In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import dolfin

In [2]:
import numpy as np
x = np.array([[1,2]]).T
A = np.array([[1,8], [3,4]])

In [3]:
#c = 'exp(1+cos(2*pi*x[0])/4.+cos(2*pi*x[1])/9.+cos(2*pi*(x[0]+x[1]))/16.+cos(4*pi*x[0])/25.+cos(4*pi*x[1])/36.)'
c = 'exp(cos(2*pi*x[0])+cos(2*pi*x[1])+cos(2*pi*(x[0]+x[1]))+cos(4*pi*x[0])+cos(4*pi*x[1]))'
#c = ' + '.join(["cos(2*pi*({}*x[0]+{}*x[1])) / {}".format(coefs[k][0], coefs[k][1], (k+1.)**2) for k in xrange(6)])
#c = 'exp({})'.format(c)
c_expr = dolfin.Expression(c)

mesh_size = 1000
mesh = dolfin.UnitSquareMesh(mesh_size, mesh_size)
fn_space = dolfin.FunctionSpace(mesh, 'Lagrange', 1)

v = dolfin.TestFunction(fn_space)
u = dolfin.TrialFunction(fn_space)

a = -dolfin.inner(dolfin.nabla_grad(v), c_expr*dolfin.nabla_grad(u))
f = dolfin.inner(v, dolfin.Constant(0))

u_sol = dolfin.Function(fn_space)

def x2_on_left_bdy(x, on_bdy):
    return on_bdy and dolfin.near(x[1], 0.)

def x2_on_right_bdy(x, on_bdy):
    return on_bdy and dolfin.near(x[1], 1.)

bcs = [
    dolfin.DirichletBC(fn_space, dolfin.Expression('x[0]'), x2_on_left_bdy),
    dolfin.DirichletBC(fn_space, dolfin.Expression('1-x[0]'), x2_on_right_bdy)
]

# neumann conditions automatically satisfied since they are homogeneous
dolfin.solve(a*dolfin.dx == f*dolfin.dx, u_sol, bcs)

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_030172ac36a54f6832c74e6833003d3326a83aa3

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
INFO:FFC:  
INFO:FFC:  Geometric dimension:       2
  Number of cell subdomains: 0
  Rank:                      1
  Arguments:                 '(v_0)'
  Number of coefficients:    0
  Coefficients:              '[]'
  Unique elements:           'CG1(?)'
  Unique sub elements:       'CG1(?)'
  
INFO:FFC:  Extracting monomial form representation from UFL form
INFO:FFC:  Transforming monomial form to reference element
DEBUG:FFC:  Estimated cost of tensor representation: 0
INFO:FFC:  representation:    auto --> tensor
DEBUG:FFC:  Selecting quadrature degree based on total polynomial degree of integrand: 1
INFO:FFC:  quadrature_degree: auto --> 1
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.065172 

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_d44ae5d8ecab507c7a57990baf92ca5573dadc22

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
INFO:UFL:Adjusting missing element domain to <Domain built from <triangle cell in 2D> with label dolfin_mesh_with_id_1>.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       2
  Number of cell subdomains: 0
  Rank:                      1
  Arguments:                 '(v_0)'
  Number of coefficients:    1
  Coefficients:              '[f_5]'
  Unique elements:           'CG1(?), R0(?)'
  Unique sub elements:       'CG1(?), R0(?)'
  
INFO:FFC:  Extracting monomial form representation from UFL form
INFO:FFC:  Transforming monomial form to reference element
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Estimated cost of tensor representation: 1
INFO:FFC:  representation:    auto --> tensor
DEBUG:FFC:  Selecting quadrature degree based on total poly

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_2293ee7bf424937bbd8550d345ee634bc2269e48

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
INFO:UFL:Adjusting missing element domain to <Domain built from <triangle cell in 2D> with label dolfin_mesh_with_id_1>.
INFO:UFL:Adjusting missing element degree to 1
INFO:FFC:  
INFO:FFC:  Geometric dimension:       2
  Number of cell subdomains: 0
  Rank:                      2
  Arguments:                 '(v_0, v_1)'
  Number of coefficients:    1
  Coefficients:              '[f_0]'
  Unique elements:           'CG1(?)'
  Unique sub elements:       'CG1(?)'
  
INFO:FFC:  Extracting monomial form representation from UFL form
INFO:FFC:  Transforming monomial form to reference element
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Estimated cost of tensor represe

In [ ]:
# for ipython notebook
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.tri as tri

def mesh2triang(mesh):
    xy = mesh.coordinates()
    return tri.Triangulation(xy[:, 0], xy[:, 1], mesh.cells())

def plot(obj):
    plt.gca().set_aspect('equal')
    if isinstance(obj, dolfin.Function):
        mesh = obj.function_space().mesh()
        if (mesh.geometry().dim() != 2):
            raise(AttributeError)
        if obj.vector().size() == mesh.num_cells():
            C = obj.vector().array()
            plt.tripcolor(mesh2triang(mesh), C)
        else:
            C = obj.compute_vertex_values(mesh)
            plt.tripcolor(mesh2triang(mesh), C, shading='gouraud')
    elif isinstance(obj, dolfin.Mesh):
        if (obj.geometry().dim() != 2):
            raise(AttributeError)
        plt.triplot(mesh2triang(obj), color='k')


In [ ]:
plt.figure(figsize=(8,8))
plot(u_sol)
plt.colorbar()
plt.gcf().patch.set_alpha(0.0)
plt.axis('scaled')

(0.0, 1.0, 0.0, 1.0)

In [ ]:
import numpy as np
X,Y = np.mgrid[0:1:6j, 0:1:6j]
x,y = X.ravel(), Y.ravel()

In [ ]:
sol_points = np.vectorize(u_sol)(x,y)

In [ ]:
np.savetxt('true_linear_elliptic_nodenom.txt', np.c_[x,y,sol_points], header='x,y,u')